In [58]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
import datetime
import psycopg2

In [59]:
pd.set_option('display.max_columns', 500)
engine = create_engine('postgresql://postgres:@OPtimus7624@localhost/digitalclass')

In [60]:
pd.read_sql(("""
    DROP VIEW IF EXISTS order_sub cascade;
    CREATE OR REPLACE VIEW order_sub 
    AS SELECT custid, orderdate, COUNT(orderno) AS n_of_order
    FROM orders
    GROUP BY custid, orderdate;
    SELECT *
    FROM order_sub
    LIMIT 5
"""), engine.connect())

,custid,orderdate,n_of_order
0,22120,2004-12-13,1
1,22120,2006-12-09,1
2,24436,2007-06-16,1
3,29278,2003-12-01,1
4,29278,2004-12-11,1


# Response Rate by Catalog

In [61]:
pd.read_sql(("""
    DROP VIEW IF EXISTS contact_c cascade;
    CREATE OR REPLACE VIEW contact_c 
    AS (SELECT custid, contactdate, COUNT(contactdate) AS catalog_sent
    FROM contacts 
    WHERE contacttype = 'C'
    GROUP BY custid, contactdate);
    SELECT * FROM contact_c LIMIT 5
"""), engine.connect())

,custid,contactdate,catalog_sent
0,22120,2005-04-14,2
1,22120,2005-09-09,1
2,22120,2005-10-14,1
3,22120,2005-11-03,1
4,22120,2005-11-10,1


In [62]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_catalog cascade;
    CREATE OR REPLACE VIEW respond_catalog 
    AS (SELECT contact_c.custid, SUM(catalog_sent) AS sum_respond_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <= 7) AND (order_sub.orderdate - contact_c.contactdate >= 0)
    GROUP BY contact_c.custid);
    SELECT *
    FROM respond_catalog
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_catalog
0,22120,1.0
1,29278,5.0
2,176320,3.0
3,333031,1.0
4,353683,1.0


## Date Difference: 7 days

In [63]:
pd.read_sql(("""
    SELECT SUM(catalog_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS ttl_catalog_sent,
    SUM(catalog_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS response_rate_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <=  7) AND (order_sub.orderdate - contact_c.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_catalog_sent,response_rate_catalog
0,19744.0,1021014,0.019338


In [64]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_catalog cascade;
    CREATE OR REPLACE VIEW respond_catalog 
    AS (SELECT contact_c.custid, SUM(catalog_sent) AS sum_respond_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <= 10) AND (order_sub.orderdate - contact_c.contactdate >= 0)
    GROUP BY contact_c.custid);
    SELECT *
    FROM respond_catalog
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_catalog
0,22120,1.0
1,29278,6.0
2,176320,4.0
3,333031,1.0
4,353683,2.0


## Date Difference: 10 days

In [65]:
pd.read_sql(("""
    SELECT SUM(catalog_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS ttl_catalog_sent,
    SUM(catalog_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS response_rate_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <=  10) AND (order_sub.orderdate - contact_c.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_catalog_sent,response_rate_catalog
0,27153.0,1021014,0.026594


In [66]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_catalog cascade;
    CREATE OR REPLACE VIEW respond_catalog 
    AS (SELECT contact_c.custid, SUM(catalog_sent) AS sum_respond_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <= 14) AND (order_sub.orderdate - contact_c.contactdate >= 0)
    GROUP BY contact_c.custid);
    SELECT *
    FROM respond_catalog
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_catalog
0,22120,1.0
1,29278,6.0
2,176320,4.0
3,262695,1.0
4,333031,2.0


## Date Difference: 14 days

In [67]:
pd.read_sql(("""
    SELECT SUM(catalog_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS ttl_catalog_sent,
    SUM(catalog_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'C') AS response_rate_catalog
    FROM contact_c
    INNER JOIN order_sub
    ON contact_c.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_c.contactdate <=  14) AND (order_sub.orderdate - contact_c.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_catalog_sent,response_rate_catalog
0,38080.0,1021014,0.037296


# Response Rate by Email

In [68]:
pd.read_sql(("""
    DROP VIEW IF EXISTS contact_e cascade;
    CREATE OR REPLACE VIEW contact_e 
    AS (SELECT custid, contactdate, COUNT(contactdate) AS email_sent
    FROM contacts 
    WHERE contacttype = 'E'
    GROUP BY custid, contactdate);
    SELECT * 
    FROM contact_e
    LIMIT 5;
"""), engine.connect()) 

,custid,contactdate,email_sent
0,24436,2006-03-21,1
1,24436,2006-04-03,1
2,24436,2006-04-06,1
3,24436,2006-04-27,1
4,24436,2006-05-04,1


In [69]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_email cascade;
    CREATE OR REPLACE VIEW respond_email 
    AS (SELECT contact_e.custid, SUM(email_sent) AS sum_respond_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  7) AND (order_sub.orderdate - contact_e.contactdate >= 0)
    GROUP BY contact_e.custid);
    SELECT *
    FROM respond_email
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_email
0,24436,1.0
1,206591,2.0
2,229873,4.0
3,361208,12.0
4,568555,6.0


## Date Difference: 7 days

In [70]:
pd.read_sql(("""
    SELECT SUM(email_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS ttl_email_sent,
    SUM(email_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS response_rate_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  7) AND (order_sub.orderdate - contact_e.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_email_sent,response_rate_email
0,46142.0,2368315,0.019483


In [71]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_email cascade;
    CREATE OR REPLACE VIEW respond_email 
    AS (SELECT contact_e.custid, SUM(email_sent) AS sum_respond_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  10) AND (order_sub.orderdate - contact_e.contactdate >= 0)
    GROUP BY contact_e.custid);
    SELECT *
    FROM respond_email
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_email
0,24436,2.0
1,206591,2.0
2,229873,5.0
3,361208,15.0
4,568555,8.0


## Date Difference: 10 days

In [72]:
pd.read_sql(("""
    SELECT SUM(email_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS ttl_email_sent,
    SUM(email_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS response_rate_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  10) AND (order_sub.orderdate - contact_e.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_email_sent,response_rate_email
0,61145.0,2368315,0.025818


In [73]:
pd.read_sql(("""
    DROP VIEW IF EXISTS respond_email cascade;
    CREATE OR REPLACE VIEW respond_email 
    AS (SELECT contact_e.custid, SUM(email_sent) AS sum_respond_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  14) AND (order_sub.orderdate - contact_e.contactdate >= 0)
    GROUP BY contact_e.custid);
    SELECT *
    FROM respond_email
    LIMIT 5
"""), engine.connect())

,custid,sum_respond_email
0,24436,3.0
1,206591,4.0
2,229873,8.0
3,361208,16.0
4,568555,9.0


## Date Difference: 14 days

In [74]:
pd.read_sql(("""
    SELECT SUM(email_sent) AS responded_catalog, 
    (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS ttl_email_sent,
    SUM(email_sent)/ (SELECT COUNT(*) FROM contacts WHERE contacttype = 'E') AS response_rate_email
    FROM contact_e
    INNER JOIN order_sub
    ON contact_e.custid = order_sub.custid
    WHERE (order_sub.orderdate - contact_e.contactdate <=  14) AND (order_sub.orderdate - contact_e.contactdate >= 0)
"""), engine.connect())

,responded_catalog,ttl_email_sent,response_rate_email
0,80293.0,2368315,0.033903
